In [ ]:
def change_duplicate_columns(columns):
    seen = []
    final = []
    for column in columns:
        if column in seen:
            final.append(column + '1')
        else:
            final.append(column)
        seen.append(column)
    return final



In [ ]:
def remove_numbers(columname):
    splitted = columname.split('_')
    if len(splitted) > 1:
        splitted = splitted[:-1]
        if len(splitted) > 1:
            return '_'.join(splitted)
    return splitted[0]

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 999
to_keep = ['WijkenEnBuurten', 'SoortRegio',
'AantalInwoners', 'Mannen', 'Vrouwen',
'k_0Tot15Jaar', 'k_15Tot25Jaar', 'k_25Tot45Jaar', 'k_45Tot65Jaar',
'k_65JaarOfOuder', 'Ongehuwd', 'Gehuwd', 'Gescheiden', 'Verweduwd',
'WestersTotaal', 'NietWestersTotaal',
'GeboorteRelatief', 'SterfteRelatief', 'HuishoudensTotaal',
'Eenpersoonshuishoudens', 'HuishoudensZonderKinderen',
'HuishoudensMetKinderen', 'GemiddeldeHuishoudensgrootte',
'Bevolkingsdichtheid', 'Woningvoorraad', 'GemiddeldeWoningwaarde',
'PercentageEengezinswoning', 'PercentageMeergezinswoning',
'PercentageBewoond', 'Koopwoningen',
'HuurwoningenTotaal', 'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
'BouwjaarVoor2000', 'BouwjaarVanaf2000',
'GemiddeldElektriciteitsverbruikTotaal', 'Appartement', 'Tussenwoning',
'Hoekwoning', 'TweeOnderEenKapWoning', 'VrijstaandeWoning', 'Huurwoning', 'GemiddeldAardgasverbruikTotaal',
'AantalInkomensontvangers', 'GemiddeldInkomenPerInkomensontvanger',
'GemiddeldInkomenPerInwoner', 'k_40PersonenMetLaagsteInkomen',
'k_20PersonenMetHoogsteInkomen', 'k_20HuishoudensMetHoogsteInkomen',
'HuishoudensMetEenLaagInkomen', 'HuishOnderOfRondSociaalMinimum',
'BedrijfsvestigingenTotaal',
'PersonenautoSTotaal', 'PersonenautoSJongerDan6Jaar',
'PersonenautoS6JaarEnOuder', 'PersonenautoSBrandstofBenzine',
'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
'PersonenautoSNaarOppervlakte', 'Motorfietsen',
'MateVanStedelijkheid','Omgevingsadressendichtheid']

buurtinfo2016 = pd.read_csv('../data/groningen/groningen_kern2016.csv',sep=';', encoding = "ISO-8859-1", skipinitialspace=True)
columns = buurtinfo2016.columns.map(remove_numbers)
buurtinfo2016.columns = columns
buurtinfo2016 = buurtinfo2016[to_keep]
buurtinfo2016['attributes.TELJAAR'] = 2016
buurtinfo2016.columns = change_duplicate_columns(buurtinfo2016.columns)

In [ ]:
ground_truth = pd.read_csv('../data/alle_jaren.csv')

In [ ]:
ground_truth = ground_truth[ground_truth['attributes.TELJAAR']==2016]

In [ ]:
ground_truth_number = ground_truth[['attributes.AANTAL_SUMMED','WijkenEnBuurten']].copy()

In [ ]:
def replace_dots(value):
    try:
        value = pd.to_numeric(value)
        return value
    except:
        return np.nan

In [ ]:
for x in buurtinfo2016.columns[2:]:
    buurtinfo2016[x] = buurtinfo2016[x].map(lambda y: replace_dots(y))

buurtinfo2016 = buurtinfo2016.sort_values(['WijkenEnBuurten', 'attributes.TELJAAR']).fillna(method='pad')
buurtinfo2016 = buurtinfo2016.sort_values(['WijkenEnBuurten', 'attributes.TELJAAR']).fillna(method='bfill')

In [ ]:
buurtinfo2016_filter = buurtinfo2016[buurtinfo2016['SoortRegio'].str.match('Buurt')]
buurtinfo2016_filter['WijkenEnBuurten'] = buurtinfo2016_filter['WijkenEnBuurten'].map(lambda x: x[4:])
buurtinfo2016_filter['WijkenEnBuurten'] = pd.to_numeric(buurtinfo2016_filter['WijkenEnBuurten'])
buurtinfo2016_filter = buurtinfo2016_filter[buurtinfo2016_filter['WijkenEnBuurten'].isin(weights['WijkenEnBuurten'])]
buurtinfo2016_filter['Zon_atlas'] = ground_truth['Zon_atlas'].values

In [ ]:
X_2016 = buurtinfo2016_filter[buurtinfo2016_filter.columns[2:]].values

In [ ]:
import lightgbm as lgb
bst = lgb.Booster(model_file='../data/model.txt')
y_pred = bst.predict(X_2016)

In [ ]:
weights = pd.read_csv("../data/buurt_weights.csv",index_col=None)

In [ ]:
weights = weights[weights['WijkenEnBuurten'].isin(buurtinfo2017_filter['WijkenEnBuurten'])]
weights['Zon_atlas'] = 0

In [ ]:
buurtinfo2016_filter = buurtinfo2016_filter.drop('SoortRegio',axis=1).drop('attributes.TELJAAR',axis=1)

In [ ]:
buurtinfo2016_filter.index = weights.index

In [ ]:
pred_2016 = y_pred
true_2016 = ground_truth_number['predicted_nr_panels']

In [ ]:
pred_series = pd.Series(pred_2016)

In [ ]:
buurtinfo2016_filter.to_csv('../data/demo/buurtinfo2016.csv')
true_2016.to_csv('../data/demo/true_labels.csv')
pred_series.to_csv('../data/demo/pred_labels.csv')

In [ ]:
def pipeline(weights, buurtinfo2016_filter, true_2016, pred_2016):
    for i in range(1,14):
        weights_only = weights.copy()
        weights_only = weights_only * i
        weights_only['WijkenEnBuurten'] = 0
        buurtinfo2016_multiplied = buurtinfo2016_filter.add(weights_only,fill_value=0)
        diff = true_2016 - pred_2016
        y_pred = bst.predict(buurtinfo2016_multiplied[buurtinfo2016_multiplied.columns[1:]].values)
        y_pred = y_pred + diff
        df_predicted = buurtinfo2016_multiplied[['WijkenEnBuurten']].copy()
        df_predicted['year'] = 2016 + i
        y_pred[y_pred<0] = 0
        y_pred = fix_negatives(y_pred, true_2016.values)
        df_predicted['predicted_nr_panels'] = y_pred
        df_predicted.to_csv('../data/predictions/predictions_{}.csv'.format(2016 + i))

In [ ]:
def fix_negatives(y_pred, true_2016):
    return_list = []
    for i, value in enumerate(y_pred):
        if value < true_2016[i]:
            return_list.append(true_2016[i])
        else:
            return_list.append(value)
    return return_list

In [ ]:
pipeline(weights, buurtinfo2016_filter, true_2016, pred_2016)

In [ ]:
ground_truth_number = ground_truth_number.rename(columns={'attributes.AANTAL_SUMMED':'predicted_nr_panels'})

In [ ]:
ground_truth_number.to_csv('../data/predictions/predictions_2016.csv')